In [13]:
import json
import cv2
import os
from scipy import misc
import shutil
from matplotlib import pyplot as plt
import numpy as np
import numpy.ma as ma
import struct
from random import shuffle
%matplotlib inline
import xml.etree.ElementTree as ET 
import json

In [14]:
name = "plate"
data_dir = "plate_keypoint" ##IMAGEPATH
raw_data_dir = "plate_keypoint_raw"##orginal xml file path

In [15]:
def parse_xml(xml_file_path,img_path_):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    images = root.find("images")
    images = images.findall("image")
    
    for image in images:
        image_name = image.get("file").split('\\')[-1]
        image_path = os.path.join(img_path_,image_name)
        image_label = []
        x =y=z=w = []
        boxes = image.findall("box")
        if len(boxes) == 0:
            print(image_path + ", this img has no labeled box!!!")
            continue
        
        elif len(boxes) == 1:
            for box in boxes:
                top = int(box.get("top"))
                left = int(box.get("left"))
                width = int(box.get("width"))
                height = int(box.get("height"))
                x = [left,top]
                y = [left+width,top]
                z = [left+width,top+height]
                w = [left,top+height]
            image_label.append(x)
            image_label.append(y) 
            image_label.append(z)  
            image_label.append(w)
            
        elif len(boxes) == 2:
            flag = False
            for box in boxes:
                top = int(box.get("top"))
                left = int(box.get("left"))
                width = int(box.get("width"))
                height = int(box.get("height"))
                if flag == False:
                    x = [left,top]
                    z = [left+width,top+height]
                    flag = True 
                else:
                    y = [left+width,top]
                    w = [left,top+height]
            image_label.append(x)
            image_label.append(y) 
            image_label.append(z)  
            image_label.append(w)    

        else:
            continue
        
        img1 = cv2.imread(os.path.join(img_path_,image_name))   
        print(image_label)
        img_name = image_name[0:int(image_name.index("."))]
        xml_file = os.path.join(img_path_, img_name + ".json")
        print(xml_file)
        if os.path.isfile(xml_file) == True:
            os.remove(xml_file)
        with open(xml_file, 'w') as json_file:
            json_file.write(str(image_label))

In [16]:
# name = "plate"
# data_dir = "plate_keypoint"
# raw_data_dir = "plate_keypoint_raw"
data_dirs = os.listdir(raw_data_dir)

In [17]:
for _data_dir in data_dirs:
    if _data_dir.endswith(".xml") == False:
        continue
#     print(os.path.join(XML_DATA_ROOT,data_dir))
    parse_xml(os.path.join(raw_data_dir,_data_dir),data_dir)
print("image transfor json finished!!!")

[[43, 22], [121, 13], [122, 40], [44, 48]]
plate_keypoint/00048fa2-25cc-4163-bbf2-11f8664c0faa.json
[[26, 8], [74, 8], [74, 24], [26, 24]]
plate_keypoint/002309e7-c657-4802-8c71-7e2530a25e3d.json
[[20, 30], [77, 22], [81, 45], [24, 53]]
plate_keypoint/005a2b45-443f-46e5-b0b1-b921aa4fdec1.json
[[15, 14], [53, 14], [53, 25], [15, 25]]
plate_keypoint/0076d2ad-6c6f-4add-8f53-933ab79c55b7.json
[[19, 6], [65, 6], [65, 21], [19, 21]]
plate_keypoint/00e3da1f-fc57-46cd-af43-36fc6b838bf7.json
[[20, 9], [73, 9], [73, 27], [20, 27]]
plate_keypoint/00ff85f6-394d-4b22-97f2-08c97d684313.json
[[24, 11], [76, 11], [76, 28], [24, 28]]
plate_keypoint/015d5ec1-b29d-46a9-930d-7983856d8b4e.json
[[55, 22], [151, 24], [151, 59], [55, 57]]
plate_keypoint/016bdb4f-0ea6-495d-950e-67c44f025b39.json
[[19, 9], [60, 9], [60, 24], [19, 24]]
plate_keypoint/017ea0f4-dcb5-4b24-8d49-fbb786739e69.json
[[35, 14], [80, 14], [80, 29], [35, 29]]
plate_keypoint/01a6ac96-dcf5-4580-a5d8-390ce10e5f53.json
[[11, 9], [64, 11], [62,

In [18]:
img_names = os.listdir(data_dir)
records = []
for img_name in img_names:
    # only fetch the original image
    if "json" in img_name:
        continue
    
    # get the image
    img_path = os.path.join(data_dir, img_name)
    img = misc.imread(img_path)
    
    img_height, img_width, _ = img.shape
    
    # verify bone json file exist
    bone_name = img_name.split('.')[0] + ".json"
    bone_path = os.path.join(data_dir, bone_name)
    if os.path.isfile(bone_path) == False:
        continue
    
    # get the bone points
    with open(bone_path) as f:
        bone_info = json.load(f)

    # normalize each bone point by the segmented box
    norm_bone = []
    for bone_idx, bone in enumerate(bone_info):
        norm_bone_x = bone[0] / img_width - 0.5
        norm_bone_y = bone[1] / img_height - 0.5
        norm_bone.append([norm_bone_x, norm_bone_y])
    
    norm_bone = [str(ele[0]) + " " + str(ele[1]) for ele in norm_bone]
    norm_bone = ' '.join(norm_bone)
    
    record = img_path + " " + norm_bone + "\n"
    
    records.append(record)

shuffle(records)
print("Number of records: " + str(len(records)))

Number of records: 866


In [19]:
# split into training set and test set
test_ratio = 0.1
total_num = len(records)
test_num = int(test_ratio * total_num)
train_num = total_num - test_num
train_records = records[0:train_num]
test_records = records[train_num:]

In [20]:
# save to text file
all_out_file = open(name + "_all.txt", "w")
for record in records:
    all_out_file.write(record)
all_out_file.close()

train_out_file = open(name + "_train.txt", "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()

test_out_file = open(name + "_test.txt", "w")
for record in test_records:
    test_out_file.write(record)
test_out_file.close()